## Serial

In [ ]:
import time
start=time.time()
def y(x):
    return (4/(1+x**2))
def trapezoid(a,b,n):
        h=(b-a)/n
        s=(y(a)+y(b))
        i=1
        for i in range (1,n):
            s+=2*y(a+i*h)
            i+=1
        return((h/2)*s)
a = 0
b = 1
n = 10**9
print("integral=",trapezoid(a,b,n))
print(time.time()-start)

integral= 3.1399259889071587
0.0


## P2P

In [ ]:
from mpi4py import MPI
import timeit
import math
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
start = MPI.Wtime()

def y(x):
    return (4/(1+x**2))

def trapezoid(a,b,n):
    h=(b-a)/n
    s=(y(a)+y(b))
    i=1
    for i in range (1,n):
        s+=2*y(a+i*h)
        i+=1
    return((h/2)*s)

a = 0
b = 1
n = 10**9
h = (b - a)/n
local_n = n // size
local_a = a + rank * local_n * h
local_b = local_a + local_n * h
local_integral = trapezoid(local_a, local_b, local_n)
#print("hasil local_integral = ", local_integral, "pada proses", rank)
my_integral=0
if rank == 0:
    my_integral = local_integral
    for i in range(1,size):
        my_integral_2 = comm.recv(source=MPI.ANY_SOURCE)
        my_integral = my_integral + my_integral_2
        print("hasil numerik = ", my_integral)
        print("waktu = ", MPI.Wtime()-start, "detik")
else:
    comm.send(local_integral,dest=0)

## Reduce

In [ ]:
from mpi4py import MPI
import timeit
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
start = MPI.Wtime()
def y(x):
    return (4/(1+x**2))
def trapezoid(a,b,n):
    h=(b-a)/n
    s=(y(a)+y(b))
    i=1
    for i in range (1,n):
        s+=2*y(a+i*h)
        i+=1
    return((h/2)*s)
a = 0
b = 1
n = 10**9
h = (b - a)/n
local_n = n // size
local_a = a + rank * local_n * h
local_b = local_a + local_n * h
local_integral = trapezoid(local_a, local_b, local_n)
#print("hasil local_integral = ", local_integral, "pada proses", rank)
my_integral=local_integral
my_integral=comm.reduce(my_integral, op=MPI.SUM, root=0)
if rank == 0:
    print("hasil numerik = ", my_integral)
    print("waktu = ", MPI.Wtime()-start, "detik")

## Broadcast-reduce

In [ ]:
from mpi4py import MPI
import timeit
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
start = MPI.Wtime()
def y(x):
    return (4/(1+x**2))

def trapezoid(a,b,n):
    h=(b-a)/n
    s=(y(a)+y(b))
    i=1
    for i in range (1,n):
        s+=2*y(a+i*h)
        i+=1
    return((h/2)*s)

a = 0
b = 1
n = 10**9

if rank == 0:
    n = 10**9
else:
    n = 0

n = comm.bcast(n,root=0)
h = (b-a)/n
local_n = n // size
local_a = a + rank * local_n * h
local_b = local_a + local_n * h
local_integral = trapezoid(local_a, local_b, local_n)

my_integral=local_integral
my_integral=comm.reduce(my_integral, op=MPI.SUM, root=0)
if rank == 0:
    print("hasil numerik = ", my_integral)
    print("waktu = ", MPI.Wtime()-start, "detik")


## Broadcast-Gather

In [ ]:
from mpi4py import MPI
import timeit
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
start = MPI.Wtime()

def y(x):
    return (4/(1+x**2))

def trapezoid(a,b,n):
    h=(b-a)/n
    s=(y(a)+y(b))
    i=1
    for i in range (1,n):
        s+=2*y(a+i*h)
        i+=1
    return((h/2)*s)

a = 0
b = 1
n = 10**9
if rank == 0:
    n = 10**9
else:
    n = 0
n = comm.bcast(n,root=0)
h = (b-a)/n
local_n = n // size
local_a = a + rank * local_n * h
local_b = local_a + local_n * h
local_integral = trapezoid(local_a, local_b, local_n)
my_integral = local_integral
my_integral = comm.gather(my_integral, root=0)

if rank == 0:
    print("hasil numerik = ", sum(my_integral))
    print("waktu = ", MPI.Wtime()-start, "detik")

## Scatter-Gather

In [ ]:
from mpi4py import MPI
import timeit
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
start = MPI.Wtime()
def y(x):
    return (4/(1+x**2))
def trapezoid(a,b,n):
    h=(b-a)/n
    s=(y(a)+y(b))
    i=1
    for i in range (1,n):
        s+=2*y(a+i*h)
        i+=1
    return((h/2)*s)
a = 0
b = 1
n = 10**9
h = (b - a)/n
i = 1
local_n = n // size
local_a = [0 for i in range (size)]
local_b = [0 for i in range (size)]
if rank == 0:
    for i in range(size):
        local_a[i]=a+rank*local_n*h
        local_b[i]=local_a[i]+local_n*h
        data = [(local_a[i],local_b[i]) for i in range(size)]
else:
    data = None
data = comm.scatter(data, root=0)
local_n = n // size
local_a = a+rank*local_n*h
local_b = local_a+local_n*h
local_integral = trapezoid(local_a, local_b, local_n)
my_integral=local_integral
my_integral=comm.gather(my_integral, root=0)
if rank == 0:
    print("hasil numerik = ", sum(my_integral))
    print("waktu = ", MPI.Wtime()-start, "detik")

## Scatter-reduce

In [ ]:
from mpi4py import MPI
import timeit
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
start = MPI.Wtime()

def y(x):
    return (4/(1+x**2))

def trapezoid(a,b,n):
    h=(b-a)/n
    s=(y(a)+y(b))
    i=1
    for i in range (1,n):
        s+=2*y(a+i*h)
        i+=1
    return((h/2)*s)

a = 0
b = 1
n = 10**9
h = (b - a)/n
i = 1
local_n = n // size
local_a = [0 for i in range (size)]
local_b = [0 for i in range (size)]
if rank == 0:
    for i in range(size):
        local_a[i]=a+rank*local_n*h
        local_b[i]=local_a[i]+local_n*h
    data = [(local_a[i],local_b[i]) for i in range(size)]
else:
    data = None
data = comm.scatter(data, root=0)
local_n = n // size
local_a = a+rank*local_n*h
local_b = local_a+local_n*h
local_integral = trapezoid(local_a, local_b, local_n)
my_integral=local_integral
my_integral=comm.reduce(my_integral, op=MPI.SUM, root=0)
if rank == 0:
    print("hasil numerik = ", my_integral)
    print("waktu = ", MPI.Wtime()-start, "detik")

## Pycuda

In [ ]:
import pycuda.autoinit
import pycuda.driver as cuda
import numpy as np
from pycuda.compiler import SourceModule
import time

kernel_code = """
__global__ void trapezoid_kernel(double a, double h, double *d_result, int n) {
    int i = threadIdx.x + blockIdx.x * blockDim.x;
    if (i >= n) return;

    double x = a + i * h;
    double y = 4.0 / (1.0 + x * x);

    d_result[i] = y;
}
"""

def trapezoid_gpu(a, b, n):
    h = (b - a) / n

    d_result = cuda.mem_alloc(n * np.dtype(np.float64).itemsize)

    mod = SourceModule(kernel_code)
    trapezoid_kernel = mod.get_function("trapezoid_kernel")

    threads_per_block = 256
    blocks_per_grid = (n + threads_per_block - 1) // threads_per_block

    trapezoid_kernel(
        np.float64(a), np.float64(h), d_result, np.int32(n),
        block=(threads_per_block, 1, 1), grid=(blocks_per_grid, 1)
    )

    h_result = np.empty(n, dtype=np.float64)
    cuda.memcpy_dtoh(h_result, d_result)

    integral = h * (0.5 * (4 / (1 + a**2) + 4 / (1 + b**2)) + np.sum(h_result))

    return integral

a = 0.0
b = 1.0
n = 10**8

start = time.time()
result = trapezoid_gpu(a, b, n)
end = time.time()

print("Integral =", result)
print("Waktu eksekusi (GPU):", end - start, "detik")

## Multiprocessing

In [ ]:
import time
import os
from multiprocessing import Pool

def y(x):
    return 4.0 / (1.0 + x*x)

def _chunk_sum(args):
    a, h, start, end = args
    s = 0.0
    for i in range(start, end):
        s += y(a + i * h)
    return s

def trapezoid_mp(a, b, n, nprocs=None):
    if n < 1:
        raise ValueError("n must be >= 1")

    h = (b - a) / n
    ya = y(a)
    yb = y(b)

    interior = n - 1
    if interior <= 0:
        return (h / 2.0) * (ya + yb)

    if nprocs is None:
        nprocs = max(1, os.cpu_count() or 1)

    chunk_size = (interior + nprocs - 1)//nprocs
    tasks = []
    start = 1
    while start <= interior:
        end = min(start + chunk_size, n)
        tasks.append((a, h, start, end))
        start = end

    with Pool(processes=nprocs) as pool:
        sum_interior = sum(pool.imap_unordered(_chunk_sum, tasks))

    return (h / 2.0) * (ya + 2.0 * sum_interior + yb)

if __name__ == "__main__":
    a = 0.0
    b = 1.0
    n = 10**9
    nprocs = None

    t0 = time.time()
    result = trapezoid_mp(a, b, n, nprocs=nprocs)
    dt = time.time() - t0

    print("integral =", result)
    print("elapsed (s) =", dt)

## Multithreading

In [ ]:
import time
import os
from concurrent.futures import ThreadPoolExecutor

def y(x):
    return 4.0 / (1.0 + x*x)

def _chunk_sum(args):
    a, h, start, end = args
    s = 0.0
    for i in range(start, end):
        s += y(a + i * h)
    return s

def trapezoid_mt(a, b, n, nthreads=None):
    if n < 1:
        raise ValueError("n harus >= 1")

    h = (b - a) / n
    ya = y(a)
    yb = y(b)

    interior = n - 1
    if interior <= 0:
        return (h / 2.0) * (ya + yb)

    if nthreads is None:
        nthreads = min(32, max(1, os.cpu_count() or 1))

    chunk_size = (interior + nthreads - 1) // nthreads
    tasks = []
    start = 1
    while start <= interior:
        end = min(start + chunk_size, n)
        tasks.append((a, h, start, end))
        start = end

    sum_interior = 0.0
    with ThreadPoolExecutor(max_workers=nthreads) as ex:
        for partial in ex.map(_chunk_sum, tasks):
            sum_interior += partial

    return (h / 2.0) * (ya + 2.0 * sum_interior + yb)

if __name__ == "__main__":
    a = 0.0
    b = 1.0
    n = 10**9
    nthreads = None

    t0 = time.time()
    result = trapezoid_mt(a, b, n, nthreads=nthreads)
    dt = time.time() - t0

    print("integral =", result)
    print("elapsed (s) =", dt) 